# Offline Engine API

SGLang provides a direct inference engine without the need for an HTTP server, especially for use cases where additional HTTP server adds unnecessary complexity or overhead. Here are two general use cases:

- Offline Batch Inference
- Custom Server on Top of the Engine

This document focuses on the offline batch inference, demonstrating four different inference modes:

- Non-streaming synchronous generation
- Streaming synchronous generation
- Non-streaming asynchronous generation
- Streaming asynchronous generation

Additionally, you can easily build a custom server on top of the SGLang offline engine. A detailed example working in a python script can be found in [custom_server](https://github.com/sgl-project/sglang/blob/main/examples/runtime/engine/custom_server.py).



## Nest Asyncio
Note that if you want to use **Offline Engine** in ipython or some other nested loop code, you need to add the following code:
```python
import nest_asyncio

nest_asyncio.apply()

```

## Advanced Usage

The engine supports [vlm inference](https://github.com/sgl-project/sglang/blob/main/examples/runtime/engine/offline_batch_inference_vlm.py) as well as [extracting hidden states](https://github.com/sgl-project/sglang/blob/main/examples/runtime/hidden_states). 

Please see [the examples](https://github.com/sgl-project/sglang/tree/main/examples/runtime/engine) for further use cases.

## Offline Batch Inference

SGLang offline engine supports batch inference with efficient scheduling.

In [1]:
# launch the offline engine
import asyncio

import sglang as sgl
import sglang.test.doc_patch
from sglang.utils import async_stream_and_merge, stream_and_merge

llm = sgl.Engine(model_path="qwen/qwen2.5-0.5b-instruct")

[2025-10-31 13:02:54] INFO utils.py:148: Note: detected 112 virtual cores but NumExpr set to maximum of 64, check "NUMEXPR_MAX_THREADS" environment variable.


[2025-10-31 13:02:54] INFO utils.py:151: Note: NumExpr detected 112 cores but "NUMEXPR_MAX_THREADS" not set, so enforcing safe limit of 16.


[2025-10-31 13:02:54] INFO utils.py:164: NumExpr defaulting to 16 threads.


[2025-10-31 13:02:54] INFO trace.py:52: opentelemetry package is not installed, tracing disabled


[2025-10-31 13:02:56] WARNING server_args.py:1144: Attention backend not explicitly specified. Use fa3 backend by default.


...

Loading safetensors checkpoint shards:   0% Completed | 0/1 [00:00<?, ?it/s]
Loading safetensors checkpoint shards: 100% Completed | 1/1 [00:00<00:00,  5.67it/s]
Loading safetensors checkpoint shards: 100% Completed | 1/1 [00:00<00:00,  5.66it/s]



Capturing batches (bs=112 avail_mem=62.17 GB):   5%|▌         | 1/20 [00:00<00:02,  6.36it/s]

Capturing batches (bs=64 avail_mem=62.14 GB):  35%|███▌      | 7/20 [00:00<00:00, 21.06it/s]

Capturing batches (bs=16 avail_mem=62.11 GB):  65%|██████▌   | 13/20 [00:00<00:00, 23.90it/s]

Capturing batches (bs=1 avail_mem=62.09 GB): 100%|██████████| 20/20 [00:00<00:00, 22.57it/s]


### Non-streaming Synchronous Generation

In [2]:
prompts = [
    "Hello, my name is",
    "The president of the United States is",
    "The capital of France is",
    "The future of AI is",
]

sampling_params = {"temperature": 0.8, "top_p": 0.95}

outputs = llm.generate(prompts, sampling_params)
for prompt, output in zip(prompts, outputs):
    print("===============================")
    print(f"Prompt: {prompt}\nGenerated text: {output['text']}")

Prompt: Hello, my name is
Generated text:  Vicky and I have been a digital marketing consultant for over 15 years. I have worked with companies like Boeing, AT&T, Standard & Poor’s, and The Smiths, and I have helped many individuals and small businesses grow their digital presence. My clients have included startups and established companies with an array of interests and pursuits including but not limited to: e-commerce, publishing, photography, and advertising. I am an experienced trainer and an active contributor to the blogging community, serving as a frequent guest on sites such as The Hemingway Review, The Daily Beast, and the Marketing Classroom blog. I am in the business of
Prompt: The president of the United States is
Generated text:  a man, but his chief advisor is a woman.
Why do you think the president of the United States is a man? The president of the United States is a man because he serves as the head of state and government of the country. He is elected to this position

### Streaming Synchronous Generation

In [3]:
prompts = [
    "Write a short, neutral self-introduction for a fictional character. Hello, my name is",
    "Provide a concise factual statement about France’s capital city. The capital of France is",
    "Explain possible future trends in artificial intelligence. The future of AI is",
]

sampling_params = {
    "temperature": 0.2,
    "top_p": 0.9,
}

print("\n=== Testing synchronous streaming generation with overlap removal ===\n")

for prompt in prompts:
    print(f"Prompt: {prompt}")
    merged_output = stream_and_merge(llm, prompt, sampling_params)
    print("Generated text:", merged_output)
    print()


=== Testing synchronous streaming generation with overlap removal ===

Prompt: Write a short, neutral self-introduction for a fictional character. Hello, my name is


Generated text:  [Name] and I'm a [Age] year old [Occupation]. I'm a [Skill] who has [Number of Years] years of experience in [Field]. I'm a [Skill] who has [Number of Years] years of experience in [Field]. I'm a [Skill] who has [Number of Years] years of experience in [Field]. I'm a [Skill] who has [Number of Years] years of experience in [Field]. I'm a [Skill] who has [Number of Years] years of experience in [Field]. I'm a [Skill] who has [Number of Years

Prompt: Provide a concise factual statement about France’s capital city. The capital of France is
Generated text:  Paris, which is known for its iconic landmarks such as the Eiffel Tower, Notre-Dame Cathedral, and the Louvre Museum. It is also a popular tourist destination, with many visitors coming to see the city's rich history and culture. Paris is a vibrant and diverse city with a rich history dating back to the Roman Empire and the French Revolution. It is home to many famous museums, including the Musée d'Orsay and the Musée 

Generated text:  likely to be characterized by rapid advancements in areas such as machine learning, natural language processing, and computer vision. Some potential trends include:

1. Increased integration of AI into everyday life: AI is already being integrated into many aspects of our lives, from self-driving cars to smart home devices. As AI becomes more integrated into our daily lives, we can expect to see even more widespread adoption.

2. AI becoming more autonomous: As AI becomes more integrated into our daily lives, we can expect to see more autonomous vehicles and robots becoming more prevalent. This could lead to a more efficient and safer way of transportation, but it could also lead to



### Non-streaming Asynchronous Generation

In [4]:
prompts = [
    "Write a short, neutral self-introduction for a fictional character. Hello, my name is",
    "Provide a concise factual statement about France’s capital city. The capital of France is",
    "Explain possible future trends in artificial intelligence. The future of AI is",
]

sampling_params = {"temperature": 0.8, "top_p": 0.95}

print("\n=== Testing asynchronous batch generation ===")


async def main():
    outputs = await llm.async_generate(prompts, sampling_params)

    for prompt, output in zip(prompts, outputs):
        print(f"\nPrompt: {prompt}")
        print(f"Generated text: {output['text']}")


asyncio.run(main())


=== Testing asynchronous batch generation ===



Prompt: Write a short, neutral self-introduction for a fictional character. Hello, my name is
Generated text:  [Your Name], and I'm a [Your Profession] with [Your Main Occupation]. I'm a [Your Profession] by training, having [Number of Years In Professional Field]. I was born in [Your Place of Birth] and have spent most of my life in [Your Country/State/Region]. What excites me the most is [What Excites You Most About Your Profession/Interest], and I strive to [What I Want To Do Next in Your Career]. I'm a [Your Profession] with [Your Main Occupation], and I'm a [Your Profession] by training, having [Number of Years In Professional

Prompt: Provide a concise factual statement about France’s capital city. The capital of France is
Generated text:  Paris, which is known for its iconic Eiffel Tower, exquisite museums, and beautiful gardens. France’s capital city is also home to the Louvre Museum, the largest and most famous art museum in the world, and the Notre-Dame Cathedral, a UNESCO W

### Streaming Asynchronous Generation

In [5]:
prompts = [
    "Write a short, neutral self-introduction for a fictional character. Hello, my name is",
    "Provide a concise factual statement about France’s capital city. The capital of France is",
    "Explain possible future trends in artificial intelligence. The future of AI is",
]

sampling_params = {"temperature": 0.8, "top_p": 0.95}

print("\n=== Testing asynchronous streaming generation (no repeats) ===")


async def main():
    for prompt in prompts:
        print(f"\nPrompt: {prompt}")
        print("Generated text: ", end="", flush=True)

        # Replace direct calls to async_generate with our custom overlap-aware version
        async for cleaned_chunk in async_stream_and_merge(llm, prompt, sampling_params):
            print(cleaned_chunk, end="", flush=True)

        print()  # New line after each prompt


asyncio.run(main())


=== Testing asynchronous streaming generation (no repeats) ===

Prompt: Write a short, neutral self-introduction for a fictional character. Hello, my name is
Generated text: 

 [

Your

 Name

],

...

 used

 to

 improve

 the

 accuracy

In [6]:
llm.shutdown()